In [1]:
cd("C:/Users/bjoer/Documents/GitHub/Power-Analyses-Psychophysics/") #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using Random
using CSV
using RData
using Statistics
using Dates

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`


R version 3.6.1 (2019-07-05) -- "Action of the Toes"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.




┌ Warning: RCall.jl: [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113


In [2]:
R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'tidyverse' was built under R version 3.6.3
│ -- Attaching packages --------------------------------------- tidyverse 1.3.0 --
│ v ggplot2 3.3.0     v purrr   0.3.3
│ v tibble  3.0.0     v stringr 1.4.0
│ v tidyr   1.0.2     v forcats 0.5.0
│ v readr   1.3.1     
│ Warning: package 'ggplot2' was built under R version 3.6.3
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' was built under R version 3.6.3
│ Warning: package 'forcats' was built under R version 3.6.3
│ -- Conflicts ------------------------------------------ tidyverse_conflicts() --
│ x dplyr

In [7]:
function SimulateDataframe(n,
        ConditionOfInterest,
        StandardValues,
        reps,
        PSE_Difference,
        JND_Difference,
        Multiplicator_PSE_Standard,
        Multiplicator_SD_Standard,
        SD_ResponseFunction,
        Mean_Variability_Between,
        SD_Variability_Between)
    
    @rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)
    
        Parameters = quickpsy(Psychometric,Difference,Answer,grouping = .(ID,ConditionOfInterest,StandardValues), bootstrap = "none")$par
        Parameters2 = Parameters %>%
        filter(parn == "p1") %>%
        select(ID,ConditionOfInterest,Mean=par, StandardValues)
        Parameters2$SD = Parameters$par[Parameters$parn == "p2"]
        FittedPsychometricFunctions = Parameters2
    
        FittedPsychometricFunctions$StandardValues = as.character(FittedPsychometricFunctions$StandardValues)
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric FittedPsychometricFunctions
    
    Psychometric[:StandardValuesAsFactor] = "placeholder"
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference+ConditionOfInterest|ID) + (Difference+ConditionOfInterest|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=true)
    
    formula2 = @formula(Mean ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelMean = fit(MixedModel,formula2, FittedPsychometricFunctions)
    
    formula3 = @formula(SD ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelSD = fit(MixedModel,formula3, FittedPsychometricFunctions)

    [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4];(coeftable(TwoLevelMean)).cols[4][2];(coeftable(TwoLevelSD)).cols[4][2]]
    
end

SimulateDataframe (generic function with 1 method)

In [ ]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [60]
Range_PSE_Difference = [-0.1,-0.05,0,0.05,0.1]
Range_JND_Difference = [-0.2,-0.1,0,0.1,0.2]
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 100
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()
n = []

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                TimeStartTrial = Dates.now()
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                Pvalues_Accuracy_TwoLevel = []
                Pvalues_Precision_TwoLevel = []
                
                for j in 1:nIterations
                Pvalues = SimulateDataframe(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    Pvalues_Accuracy_TwoLevel = [Pvalues_Accuracy_TwoLevel;Pvalues[3]]
                    Pvalues_Precision_TwoLevel = [Pvalues_Precision_TwoLevel;Pvalues[4]]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        Duration = ((Dates.now()) - TimeStartTrial))   

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        Duration=((Dates.now()) - TimeStartTrial))
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end
                
                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", PSE_Difference, " ", JND_Difference, " ", mean(Pvalues_Accuracy .< 0.05), " ", 
                    mean(Pvalues_Precision .< 0.05), " ", PowerfulDataframe[!,:Duration][CurrentRunthrough], " END. ")

            end
            CSV.write(join([reps,"_", n, "_", PSE_Difference, "_", JND_Difference, ".csv"]),PowerfulDataframe)
        end
    end
end



┌ Warning: Implicit broadcasting to a new column in DataFrame assignment is deprecated.Use `df[!, col_ind] .= v`  when `df` has some columns
│   caller = SimulateDataframe(::Int64, ::Array{Int64,1}, ::Array{Int64,1}, ::Int64, ::Float64, ::Float64, ::Int64, ::Float64, ::Float64, ::Float64, ::Float64) at In[12]:42
└ @ Main .\In[12]:42


|==============================                        | 56% ~3 s remaining     UGH 2 out of 30: 12 60 -0.3 -0.5 1.0 1.0 335807 milliseconds END. RUNTHROUGH 3 out of 30: 14 60 -0.3 -0.5 1.0 1.0 396273 milliseconds END.                                                                   RUNTHROUGH 4 out of 30: 16 60 -0.3 -0.5 1.0 1.0 444242 milliseconds END.                                                                                                                                                         RUNTHROUGH 5 out of 30: 18 60 -0.3 -0.5 0.99 1.0 507121 milliseconds END.                                                                                                                                                                                                                                                     RUNTHROUGH 6 out of 30: 20 60 -0.3 -0.5 1.0 1.0 567850 milliseconds END. RUNTHROUGH 7 out of 30: 10 60 -0.3 -0.25 1.0 0.98 238278 milliseconds END. RUNTHROUGH 8 out of 30: 

Excessive output truncated after 524334 bytes.

"C:\\Users\\bjoer\\Documents\\GitHub\\Power-Analyses-Psychophysics"

"In[24]"

"In[21]"

In [4]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
reps = [60]
PSE_Difference = -0.1
JND_Difference = 0.1
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 100
n = 10

10

In [13]:
@rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)

    require(quickpsy)
        Parameters = quickpsy(Psychometric,Difference,Answer,grouping = .(ID,ConditionOfInterest,StandardValues), bootstrap = "none")$par
        Parameters2 = Parameters %>%
        filter(parn == "p1") %>%
        select(ID,ConditionOfInterest,Mean=par, StandardValues)
        Parameters2$SD = Parameters$par[Parameters$parn == "p2"]
        FittedPsychometricFunctions = Parameters2
        
        FittedPsychometricFunctions$StandardValues = as.character(FittedPsychometricFunctions$StandardValues)
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric FittedPsychometricFunctions
    
    Psychometric[:StandardValuesAsFactor] = "placeholder"

#    for i = 1:length(Psychometric[!,:StandardValues])
#        Psychometric[i,:StandardValuesAsFactor] = string(Psychometric[i,:StandardValues])
#    end
#    
#    Psychometric[!,:StandardValuesAsFactor] = CategoricalArray(Psychometric[!,:StandardValuesAsFactor])

#    FittedPsychometricFunctions[:StandardValuesAsFactor] = "placeholder"
    
#    for i = 1:length(FittedPsychometricFunctions[!,:StandardValues])
#        FittedPsychometricFunctions[i,:StandardValuesAsFactor] = string(FittedPsychometricFunctions[i,:StandardValues])
#    end
    
#    FittedPsychometricFunctions[!,:StandardValuesAsFactor] = CategoricalArray(FittedPsychometricFunctions[!,:StandardValuesAsFactor])

    
    #formulaAccuracy = @formula(Answer ~ 1 + ConditionOfInterest + (1 + Difference|ID) + (1 + Difference|StandardValuesAsFactor));
    #modelAccuracy = GeneralizedLinearMixedModel(formulaAccuracy, Psychometric, Bernoulli())
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference+ConditionOfInterest|ID) + (Difference+ConditionOfInterest|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=true)
    
    formula2 = @formula(Mean ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelMean = fit(MixedModel,formula2, FittedPsychometricFunctions)
    
    formula3 = @formula(SD ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelSD = fit(MixedModel,formula3, FittedPsychometricFunctions)

    [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4];(coeftable(TwoLevelMean)).cols[4][2];(coeftable(TwoLevelSD)).cols[4][2]]


|======================================================|100% ~0 s remaining     

┌ Warning: Implicit broadcasting to a new column in DataFrame assignment is deprecated.Use `df[!, col_ind] .= v`  when `df` has some columns
│   caller = top-level scope at In[13]:30
└ @ Core In[13]:30


4-element Array{Float64,1}:
 1.1488551812824994e-13
 0.044658376503681395
 2.9844962039908864e-19
 0.11911260258703839

In [12]:
GLMM

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  Answer ~ 1 + Difference + ConditionOfInterest + Difference & ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 1875.4088

Variance components:
                      Column           Variance      Std.Dev.     Corr.
ID             (Intercept)          2.7541544289059 1.6595645299
               Difference           0.0707911905556 0.2660661394 -0.29
               ConditionOfInterest  0.2014407166303 0.4488214752 -1.00  0.26
StandardValues (Intercept)          0.0000016684906 0.0012917007
               Difference           0.3593757305821 0.5994795498  0.22
               ConditionOfInterest  0.0009214977617 0.0303561816  0.18  1.00

 Number of obs: 2176; levels of grouping factors: 10, 2

Fixed-effects parameters:
───────────────────────────────────────────────────────────────

In [18]:
TwoLevelMean

Linear mixed model fit by maximum likelihood
 Mean ~ 1 + ConditionOfInterest + (1 | ID) + (1 | StandardValuesAsFactor)
     logLik        -2 logLik          AIC             BIC       
  5.98111566×10³ -1.19622313×10⁴ -1.19522313×10⁴  -1.1916621×10⁴

Variance components:
                          Column     Variance   Std.Dev.  
ID                     (Intercept)  0.326686181 0.57156468
StandardValuesAsFactor (Intercept)  2.180685079 1.47671428
Residual                            0.015652497 0.12510994
 Number of obs: 9155; levels of grouping factors: 10, 2

  Fixed-effects parameters:
───────────────────────────────────────────────────────────────
                        Estimate   Std.Error   z value  P(>|z|)
───────────────────────────────────────────────────────────────
(Intercept)          6.3537       1.05972     5.99562    <1e-8
ConditionOfInterest  0.000688873  0.00261543  0.263388   0.7923
───────────────────────────────────────────────────────────────

In [28]:
Psychometric[!,:ID][1]

CategoricalString{UInt32} "s1"

4-element Array{Float64,1}:
 3.7046258636649417e-13
 0.0005415493655042497
 0.7922518980824794
 0.0

In [ ]:
n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction

In [33]:
@__DIR__

"C:\\Users\\bjoer"